# Bài toán

> **Thu thập dữ liệu báo Việt Nam (Vietnamnet - Bài tập)**


> Hoàn thành bằng cách điền vào phần #### CODE theo hướng dẫn

    ```
    #### CODE
    ####
    ```


Cũng giống như VNExpress - Demo

Mục tiêu:

- Hiểu được quy trình thu thập dữ liệu từ các trang báo Việt Nam
- Thu thập dữ liệu (bài báo) từ các trang báo Việt Nam để làm dữ liệu cho các bước xử lý sau

Đầu ra:

-  Tập file JSON chứa các bài bài báo có các trường dữ liệu:

    - `url`: link dẫn đến bài báo
    - `title`: tiêu đề bài báo
    - `description`: tóm tắt bài báo
    - `content`: nội dung bài báo
    - `metadata`: trường dữ liệu bổ sung

        - `cat`: thể loại bài báo
        - `subcat`: thể loại con của bài báo
        - `published_date`: thời gian xuất bản
        - `author`: người viết
    
- Ví dụ về một bài báo:

    ```
     {
        "url": "https://vietnamnet.vn/khi-gap-van-xui-hay-nhin-3-...",
        "title": "Khi gặp vận xui, hãy nhìn 3 bức ảnh này sẽ thấy cuộc đời ...",
        "description": "Cuộc đời như thế nào tùy thuộc vào góc nhìn của từng ...",
        "content": "Bức tranh 1: Có 3 hay 4 khúc gỗ?\nCâu trả lời là 3 hoặc 4 khúc gỗ ...",
        "metadata": {
            "cat": "ĐỜI SỐNG",
            "subcat": "GIỚI TRẺ",
            "published_date": 1721274540,
            "author": "Phan Hằng"
        },
    },
    ```


# Các bước tiến hành

## Chuẩn bị các thư viện cần thiết

In [ ]:
# Suitable for Google Colab, for local please follow the external instructions and ignore this line
# and follows https://docs.google.com/document/d/14jK9d6KHJYX0b-gFAVqAghUxT7OLAM0nP2IovL7_Rjs/edit?usp=sharing
!apt install -qq chromium-chromedriver

In [1]:
# Install selenium
!pip install -qq selenium

# Tạo thư mục để chứa data
!mkdir -p data

In [2]:
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

# other imports
import os
import json

In [3]:
# selenium setups
## https://www.tutorialspoint.com/selenium/selenium_webdriver_chrome_webdriver_options.htm

chrome_options = webdriver.ChromeOptions()

# chrome_options.add_argument('--headless') # must options for Google Colab
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")


In [4]:
MAGAZINE_NAME = "vietnamnet"
HOME_PAGE = "https://vietnamnet.vn/"

## Thu thập dữ liệu

> **Các bước thu thập bài báo**


1. News categories: Thu thập tất cả các thể loại báo của website
2. News urls: Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó
3. News articles: Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước

### Thu thập thể loại bài báo của website: Crawling categories

> **Các bước thu thập**

1. Vào trang chủ của báo
2. Thu thập các thể loại ở mục menu


In [9]:
driver = webdriver.Chrome(options=chrome_options)
# Vào trang web chính, mặc định phải chờ toàn bộ trang webload mới xong
driver.get(HOME_PAGE)

In [13]:
# Chọn and click menu buttons

#### CODE
menu_button = driver.find_element(by=By.CLASS_NAME, value='icon-hamburger')

####
menu_button.click()

In [19]:
# Lấy hết tất cả thể loại từ đây
# bỏ qua logo-premium, logo-htvn

#### CODE
cats = []

header_menu_list = driver.find_element(by=By.CLASS_NAME, value='header_submenu-content-list')
cat_menus = header_menu_list.find_elements(by=By.CSS_SELECTOR, value='li[data-i]')

for cat_menu in cat_menus:
    cat = cat_menu.find_element(by=By.TAG_NAME, value='a').get_attribute('title').strip()
    href = cat_menu.find_element(by=By.TAG_NAME, value='a').get_attribute('href').strip()
    
    cats.append({'cat_name': cat, 'url': href})
####

In [21]:
cats, len(cat_menus)

([{'cat_name': 'Chính trị', 'url': 'https://vietnamnet.vn/chinh-tri'},
  {'cat_name': 'Thời sự', 'url': 'https://vietnamnet.vn/thoi-su'},
  {'cat_name': 'Kinh doanh', 'url': 'https://vietnamnet.vn/kinh-doanh'},
  {'cat_name': 'Dân tộc và Tôn giáo',
   'url': 'https://vietnamnet.vn/dan-toc-ton-giao'},
  {'cat_name': 'Thể thao', 'url': 'https://vietnamnet.vn/the-thao'},
  {'cat_name': 'Giáo dục', 'url': 'https://vietnamnet.vn/giao-duc'},
  {'cat_name': 'Thế giới', 'url': 'https://vietnamnet.vn/the-gioi'},
  {'cat_name': 'Đời sống', 'url': 'https://vietnamnet.vn/doi-song'},
  {'cat_name': 'Văn hóa - Giải trí',
   'url': 'https://vietnamnet.vn/van-hoa-giai-tri'},
  {'cat_name': 'Sức khỏe', 'url': 'https://vietnamnet.vn/suc-khoe'},
  {'cat_name': 'Công nghệ', 'url': 'https://vietnamnet.vn/cong-nghe'},
  {'cat_name': 'Pháp luật', 'url': 'https://vietnamnet.vn/phap-luat'},
  {'cat_name': 'Ô tô xe máy', 'url': 'https://vietnamnet.vn/oto-xe-may'},
  {'cat_name': 'Du lịch', 'url': 'https://vietn

In [22]:
driver.close()

### Thu thập một số đường dẫn dựa vài từng thể loại báo của website đó: News urls


> **Cách thu thập**

Từ những thể loại và đường link tương ứng, ta lần lượt vào từng thể loại và lấy đường dẫn của các bài báo trong mục đó của trang web đó.



#### Cài đặt thông số

In [23]:
# Đặt số lượng đường dẫn cần lấy trong mỗi thể loại báo
NUM_ARTICLES_PER_CAT = 30 # có thể tăng lên

# Đường dẫn lưu data của vnexpress
DATA_URL_FILE = "data/vietnamnet_url.json"

EXCLUDING_CATEGORIES = ["Giảm nghèo bền vững", "English", "Toàn văn"] # Exclude

# Bổ sung cài đặt chromedriver
## Ta đặt load stategy ở đây là normal: https://www.selenium.dev/documentation/webdriver/drivers/options/
chrome_options.page_load_strategy = "normal"

#### Chạy thử nghiệm



In [98]:
driver = webdriver.Chrome(options=chrome_options)

In [99]:
# Vào thử một url
driver.get(cats[0]['url'])

In [40]:
# Get all titles item

#### CODE
title_news = driver.find_elements(by=By.CSS_SELECTOR, value='[class*="topStory-"]')
####

In [51]:
# Lấy đường dẫn

#### CODE
url_new = title_news[0].find_element(by=By.TAG_NAME, value='a').get_attribute('href')
url_new
####

'https://vietnamnet.vn/tong-bi-thu-goi-y-mo-hinh-chong-co-don-cho-nguoi-gia-sang-don-di-chieu-dua-ve-2443036.html'

In [52]:
url_news = []

for title_new in title_news:
    articles = title_new.find_elements(by=By.TAG_NAME, value='a')
    for article in articles:
        url_new = article.get_attribute('href')
        url_news.append(url_new)
        
url_news

['https://vietnamnet.vn/tong-bi-thu-goi-y-mo-hinh-chong-co-don-cho-nguoi-gia-sang-don-di-chieu-dua-ve-2443036.html',
 'https://vietnamnet.vn/chinh-tri/su-kien',
 'https://vietnamnet.vn/tong-bi-thu-goi-y-mo-hinh-chong-co-don-cho-nguoi-gia-sang-don-di-chieu-dua-ve-2443036.html',
 'https://vietnamnet.vn/tong-bi-thu-goi-y-mo-hinh-chong-co-don-cho-nguoi-gia-sang-don-di-chieu-dua-ve-2443036.html#comment',
 'https://vietnamnet.vn/ong-nguyen-thanh-nghi-gia-dien-minh-bach-tang-quyen-lua-chon-noi-cung-cap-2442999.html',
 'https://vietnamnet.vn/chinh-tri/su-kien',
 'https://vietnamnet.vn/ong-nguyen-thanh-nghi-gia-dien-minh-bach-tang-quyen-lua-chon-noi-cung-cap-2442999.html',
 'https://vietnamnet.vn/ong-nguyen-thanh-nghi-gia-dien-minh-bach-tang-quyen-lua-chon-noi-cung-cap-2442999.html#comment',
 'https://vietnamnet.vn/tong-bi-thu-va-chuyen-chong-co-don-cho-nguoi-cao-tuoi-sang-don-di-chieu-dua-ve-2443265.html',
 'https://vietnamnet.vn/chinh-tri/ban-luan',
 'https://vietnamnet.vn/tong-bi-thu-va-chuy

In [57]:
for index, url_new in enumerate(url_news):
    if not url_new.startswith(HOME_PAGE):
        print(index)

In [100]:
# Tìm next page

#### CODE
next_page = driver.find_element(by=By.CSS_SELECTOR, value='li.pagination-next a')
next_url = next_page.get_attribute('href')
####

In [101]:
next_url

'https://vietnamnet.vn/chinh-tri-page1'

In [64]:
driver.close()

#### Chạy thật

In [103]:
driver = webdriver.Chrome(options=chrome_options)

In [108]:
# Global variables for filtering deduplicating urls
crawled_urls = set()

def crawl_each_category_url(driver, category_url):
    """
    Functions cho lấy urls cho từng category sau khi thử nghiệm
    """
    # Get all item-news class item
    all_urls = set()
    url = category_url

    #### CODE HERE
    while len(all_urls) < NUM_ARTICLES_PER_CAT:
        driver.get(url)
        title_news = driver.find_elements(by=By.CSS_SELECTOR, value='[class*="topStory-"]')
        for title in title_news:
            articles = title.find_elements(by=By.TAG_NAME, value='a')
            for article in articles:
                try:
                    url_new = article.get_attribute('href')
                    if url_new.startswith(HOME_PAGE) and url_new not in crawled_urls:   # avoid repetition
                        all_urls.add(url_new)
                        crawled_urls.add(url_new)   # avoid repetition
                        
                except StaleElementReferenceException:
                    continue
                except NoSuchElementException:
                    print(f'NoSuchElementException at {url}')
                    
        try:
            next_page = driver.find_element(by=By.CSS_SELECTOR, value='li.pagination-next a')
            url = next_page.get_attribute('href')
        except:
            break
                    
    ####

    return all_urls

In [109]:
saved_cats = {}

# Thu thập cho từng thể loại
for cat in cats:
    cat_name = cat["cat_name"]
    url = cat['url']
    if cat_name not in EXCLUDING_CATEGORIES:
        print(f"You are at {cat}.")
        urls = crawl_each_category_url(driver, url)
        saved_cats[cat_name] = list(urls)

with open(DATA_URL_FILE, "w", encoding='utf-8') as fOut:
    json.dump(saved_cats, fOut, ensure_ascii=False, indent=4)

driver.close()

You are at {'cat_name': 'Chính trị', 'url': 'https://vietnamnet.vn/chinh-tri'}.
You are at {'cat_name': 'Thời sự', 'url': 'https://vietnamnet.vn/thoi-su'}.
You are at {'cat_name': 'Kinh doanh', 'url': 'https://vietnamnet.vn/kinh-doanh'}.
You are at {'cat_name': 'Dân tộc và Tôn giáo', 'url': 'https://vietnamnet.vn/dan-toc-ton-giao'}.
You are at {'cat_name': 'Thể thao', 'url': 'https://vietnamnet.vn/the-thao'}.
You are at {'cat_name': 'Giáo dục', 'url': 'https://vietnamnet.vn/giao-duc'}.
You are at {'cat_name': 'Thế giới', 'url': 'https://vietnamnet.vn/the-gioi'}.
You are at {'cat_name': 'Đời sống', 'url': 'https://vietnamnet.vn/doi-song'}.
You are at {'cat_name': 'Văn hóa - Giải trí', 'url': 'https://vietnamnet.vn/van-hoa-giai-tri'}.
You are at {'cat_name': 'Sức khỏe', 'url': 'https://vietnamnet.vn/suc-khoe'}.
You are at {'cat_name': 'Công nghệ', 'url': 'https://vietnamnet.vn/cong-nghe'}.
You are at {'cat_name': 'Pháp luật', 'url': 'https://vietnamnet.vn/phap-luat'}.
You are at {'cat_na

In [117]:
len(crawled_urls)

911

### Thu thập và xử lý từng bài báo dựa vào đường dẫn của bước trước: News articles


> **Cách thu thập**

Từ đường dẫn ở trong phần trước, ta lần lượt vào từng đường link đó và thu thập thông tin về bài báo.

#### Cài đặt thông số

In [119]:
# Filepath cho cái trước
FILE_URL_PATH = "data/vietnamnet_url.json"

# Đặt limit số articles lấy từ mỗi thể loại
MAX_ARTICLES_PER_CAT = 2 # Nếu set = None thì là tất cả các urls ở file trước

# Data output, mỗi thể loại là 1 file json chứa articles
DATA_FOLDER_OUTPUT = "data/vietnamnet"

# !mkdir -p $DATA_FOLDER_OUTPUT
from pathlib import Path
Path(DATA_FOLDER_OUTPUT).mkdir(parents=True, exist_ok=True)

# Để loading strategy về eager load nhanh, không quan tâm ảnh
chrome_options.page_load_strategy = "eager"

In [120]:
# Đọc file url
with open(FILE_URL_PATH, "r", encoding='utf-8') as fIn:
    url_data = json.load(fIn)

len(url_data)

23

#### Chạy thử nghiệm

In [150]:
driver = webdriver.Chrome(options=chrome_options)

# Một số url để thử nghiệm

SAMPLE_ARTICLE_URLS = [
    "https://vietnamnet.vn/bat-giam-nguyen-thu-truong-bo-tai-nguyen-va-moi-truong-nguyen-linh-ngoc-2304574.html",
    "https://vietnamnet.vn/ky-su-cong-nghe-ve-que-nuoi-oc-thu-nhe-lai-nua-ty-moi-nam-2304196.html",
    "https://vietnamnet.vn/nsnd-thanh-hoa-va-nsnd-le-dung-hai-nguoi-thay-dac-biet-cua-nsut-viet-hoan-2304214.html",
    #author in last
    "https://vietnamnet.vn/doi-giay-cu-mon-giua-gia-lanh-chau-au-cua-tong-bi-thu-nguyen-phu-trong-2303988.html",
    #class maincontent, table ignore
    "https://vietnamnet.vn/lo-dien-cao-toc-dep-bac-nhat-dat-nuoc-chay-xuyen-rung-ngap-man-2303643.html",
]

# chọn url
SAMPLE_ARTICLE_URL = SAMPLE_ARTICLE_URLS[0]

In [151]:
driver.get(SAMPLE_ARTICLE_URL)

In [152]:
# Tìm kiếm title

#### CODE HERE
driver.find_element(by=By.CSS_SELECTOR, value='h1.content-detail-title').text
####


'Bắt giam nguyên Thứ trưởng Bộ Tài nguyên và Môi trường Nguyễn Linh Ngọc'

In [153]:
# Tìm kiếm description

#### CODE HERE
driver.find_element(by=By.CSS_SELECTOR, value='h2.content-detail-sapo.sm-sapo-mb-0').text
####


'Cục Cảnh sát điều tra tội phạm về tham nhũng, kinh tế, buôn lậu (Bộ Công an) vừa bắt giam ông Nguyễn Linh Ngọc, nguyên Thứ trưởng Bộ TN&MT để điều tra tội Vi phạm quy định về quản lý, sử dụng tài sản Nhà nước gây thất thoát, lãng phí.'

In [163]:
# Thu thập thể loại

#### CODE HERE
lis_cat = driver.find_elements(
    By.CSS_SELECTOR,
    "div.bread-crumb-detail.sm-show-time > ul > li:not(:first-child)"
)
main_cat = lis_cat[0].text if len(lis_cat) > 0 else None
sub_cat = lis_cat[1].text if len(lis_cat) > 1 else None
main_cat, sub_cat
####

('THỜI SỰ', None)

In [170]:
#Thu thập ngày
#### CODE HERE
publish_date = driver.find_element(by=By.CSS_SELECTOR, value='div.header__clock.mobile span.header__clock-content').text.strip()
publish_date
####

'Thứ tư, 17/09/2025'

In [175]:
# Tìm kiếm contents
#### CODE HERE
article = driver.find_element(by=By.CLASS_NAME, value='maincontent.main-content')
children = article.find_elements(by=By.XPATH, value='./*')
####

In [ ]:
# Thu thập contents
contents = []
author = 'Unknown'

for idx, child in enumerate(children):
    text = child.text.strip()
    if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph

        #### CODE HERE
        if len(text):
            contents.append(f'[{text}]')
        ####

    elif child.tag_name == "figure" :
        ## If length > 100  --> not a caption, it's next description
        #### CODE HERE
        if len(text):
            if len(text) <= 100:
                contents.append(f'[{text}]')
            else:
                contents.append(text)
        ####

    elif child.tag_name == "table": # Do nothing rightnow
        pass

In [181]:
# Thu thập tác giả
is_author_maybe_last = False
try:
    #### CODE HERE
    author = driver.find_element(by=By.XPATH, value="//*[contains(@class, 'author__info')]").text.strip()
    ####
except NoSuchElementException:
    is_author_maybe_last = True

# Nếu không tìm thấy tác giả bên đầu --> tác giả ở cuối content nếu content cuối len < 30
if is_author_maybe_last:
    if len(contents) and len(contents[-1]) < 30:
        author = contents.pop()
    else:
        author = "Uknown"

In [182]:
author

'T.Nhung'

In [185]:
contents

['[Cục Cảnh sát điều tra tội phạm về tham nhũng, kinh tế, buôn lậu (Bộ Công an) đang điều tra vụ án “Vi phạm quy định về nghiên cứu, thăm dò, khai thác tài nguyên; Vi phạm quy định về kế toán gây hậu quả đặc biệt nghiêm trọng; Buôn lậu; Tiêu thụ tài sản do người khác phạm tội mà có; Vi phạm quy định về quản lý, sử dụng tài sản Nhà nước gây thất thoát, lãng phí” xảy ra tại Công ty CP Tập đoàn Thái Dương và các đơn vị liên quan.]',
 '[Căn cứ kết quả mở rộng điều tra vụ án, ngày 22/7/2024, Cục Cảnh sát điều tra tội phạm về tham nhũng, kinh tế, buôn lậu ra quyết định khởi tố bị can, lệnh bắt bị can để tạm giam, lệnh khám xét đối với 5 bị can về tội “Vi phạm quy định về quản lý, sử dụng tài sản Nhà nước gây thất thoát, lãng phí”, quy định tại Khoản 3 Điều 219 Bộ luật Hình sự năm 2015.]',
 '[Các bị can bị khởi tố. Ảnh: Bộ Công an]',
 '[Các bị can gồm: Nguyễn Linh Ngọc, nguyên Thứ trưởng Bộ Tài nguyên và Môi trường; Nguyễn Văn Thuấn, nguyên Tổng cục trưởng Tổng cục Địa chất và Khoáng sản Việt

In [183]:
driver.close()

#### Chạy thật

In [193]:
def get_content_metadata(driver, article_url):

    """
    Extracts and returns metadata and content from a given article URL.

    :param driver: Selenium WebDriver instance.
    :param article_url: URL of the article to extract data from.
    :return: Dictionary containing article metadata and content.
    """

    # Get to current article
    driver.get(article_url)

    # Thu thập title
    #### CODE HERE
    title = driver.find_element(by=By.CSS_SELECTOR, value='h1.content-detail-title').text
    ####

    # Thu thập description
    #### CODE HERE
    description = driver.find_element(by=By.CSS_SELECTOR, value='h2.content-detail-sapo.sm-sapo-mb-0').text
    ####

    # Thu thập thể loại
    #### CODE HERE
    lis_cat = driver.find_elements(
    By.CSS_SELECTOR,
    "div.bread-crumb-detail.sm-show-time > ul > li:not(:first-child)"
    )
    main_cat = lis_cat[0].text if len(lis_cat) > 0 else None
    sub_cat = lis_cat[1].text if len(lis_cat) > 1 else None
    ####


    # Thu thập published date
    #### CODE HERE
    publish_date = driver.find_element(by=By.CSS_SELECTOR, value='div.header__clock.mobile span.header__clock-content').text.strip()
    ####


    # Thu thập content bài báo
    #### CODE HERE
    
    article = driver.find_element(by=By.CLASS_NAME, value='maincontent.main-content')
    children = article.find_elements(by=By.XPATH, value='./*')

    contents = []
    author = 'Unknown'

    for idx, child in enumerate(children):
        text = child.text.strip()
        if child.tag_name in ["p", "h1", "h2", "h3", "h4"]: # paragraph

            #### CODE HERE
            if len(text):
                contents.append(f'[{text}]')
            ####

        elif child.tag_name == "figure" :
            ## If length > 100  --> not a caption, it's next description
            #### CODE HERE
            if len(text):
                if len(text) <= 100:
                    contents.append(f'[{text}]')
                else:
                    contents.append(text)
            ####

        elif child.tag_name == "table": # Do nothing rightnow
            pass
    ####

    # Thu thập tác giả
    #### CODE HERE
    is_author_maybe_last = False
    try:
        #### CODE HERE
        author = driver.find_element(by=By.XPATH, value="//*[contains(@class, 'author__info')]").text.strip()
        ####
    except NoSuchElementException:
        is_author_maybe_last = True

    # Nếu không tìm thấy tác giả bên đầu --> tác giả ở cuối content nếu content cuối len < 30
    if is_author_maybe_last:
        if len(contents) and len(contents[-1]) < 30:
            author = contents.pop()
        else:
            author = "Uknown"
    ####


    return {
        "url": article_url,
        "title": title,
        "description": description,
        "content": "\n".join(contents),
        "metadata": {
            "cat": main_cat,
            "subcat": sub_cat,
            "published_date": publish_date,
            "author": author
        }
    }


In [195]:
driver = webdriver.Chrome(options=chrome_options)

for cat, urls in url_data.items():

    print(f"Thu thập dữ liệu thể loại {cat} ..")
    count_crawled = 0
    cat_data = []
    for url in urls:
        try:
            cat_data.append(get_content_metadata(driver, url))
            count_crawled += 1
            if MAX_ARTICLES_PER_CAT and count_crawled >= MAX_ARTICLES_PER_CAT:
                break

        except (StaleElementReferenceException) as e:
            print(f"Bug at url: {url}, with StaleElementReferenceException")
            driver.refresh()
            continue

        except (NoSuchElementException) as e:
            print(f"Bug at url: {url}, with NoSuchElementException")
            driver.refresh()
            continue


    name_file_cat = cat.lower().replace(" ", "-") + ".json"
    with open(os.path.join(DATA_FOLDER_OUTPUT, name_file_cat), "w", encoding='utf-8') as fOut:
        json.dump(cat_data, fOut, ensure_ascii=False, indent=4)

driver.close()

# Bug will be emagazines and videos, we ignore them as well

Thu thập dữ liệu thể loại Chính trị ..
Bug at url: https://vietnamnet.vn/chinh-tri/ban-luan, with NoSuchElementException
Thu thập dữ liệu thể loại Thời sự ..
Thu thập dữ liệu thể loại Kinh doanh ..
Bug at url: https://vietnamnet.vn/kinh-doanh/doanh-nhan, with NoSuchElementException
Thu thập dữ liệu thể loại Dân tộc và Tôn giáo ..
Thu thập dữ liệu thể loại Thể thao ..
Bug at url: https://vietnamnet.vn/the-thao/tin-chuyen-nhuong, with NoSuchElementException
Thu thập dữ liệu thể loại Giáo dục ..
Thu thập dữ liệu thể loại Thế giới ..
Thu thập dữ liệu thể loại Đời sống ..
Bug at url: https://vietnamnet.vn/doi-song, with NoSuchElementException
Thu thập dữ liệu thể loại Văn hóa - Giải trí ..
Thu thập dữ liệu thể loại Sức khỏe ..
Bug at url: https://vietnamnet.vn/suc-khoe/tri-DS40001U2.html#vnn_source=suc-khoe&vnn_medium=box_tracuubenh4, with NoSuchElementException
Bug at url: https://vietnamnet.vn/suc-khoe/benh, with NoSuchElementException
Thu thập dữ liệu thể loại Công nghệ ..
Thu thập dữ li

In [196]:
# Xem 1 sample
cat_data[-1]

{'url': 'https://vietnamnet.vn/cong-cu-ai-len-ngoi-website-van-giu-vi-tri-ngoi-nha-so-cua-doanh-nghiep-2434899.html',
 'title': 'Công cụ AI lên ngôi, website vẫn giữ vị trí “ngôi nhà số” của doanh nghiệp',
 'description': 'Các công cụ AI đang thay đổi cách người dùng tiếp cận thông tin trên Internet. Song trong bức tranh kinh doanh online, website tên miền riêng vẫn là “ngôi nhà số”, nơi doanh nghiệp tạo dựng niềm tin, thương hiệu và thúc đẩy giao dịch.',
 'content': '[Thời gian vừa qua, sự xuất hiện của ChatGPT và các công cụ trí tuệ nhân tạo - AI đã tạo ra một bước ngoặt lớn trong hành vi tìm kiếm của người dùng Internet.]\n[Thay vì chọn lọc thông tin từ loạt kết quả được công cụ hỗ trợ tìm kiếm như Google đưa ra theo từ khóa như trước đây, hiện nay người dùng chỉ cần đặt câu hỏi cụ thể cho ChatGPT là có toàn bộ thông tin được tổng hợp sẵn, trình bày gọn gàng, có ngữ cảnh và thậm chí mang tính tư vấn.]\n[Sự thay đổi trên khiến cho trải nghiệm tìm kiếm thông tin trở nên cá nhân hóa hơ

In [197]:
for key in url_data.keys():
    print(f'{key} - {len(url_data[key])}')

Chính trị - 47
Thời sự - 51
Kinh doanh - 30
Dân tộc và Tôn giáo - 41
Thể thao - 38
Giáo dục - 33
Thế giới - 43
Đời sống - 34
Văn hóa - Giải trí - 48
Sức khỏe - 38
Công nghệ - 32
Pháp luật - 40
Ô tô xe máy - 46
Du lịch - 43
Bất động sản - 40
Bạn đọc - 38
Tuần Việt Nam - 51
Công nghiệp hỗ trợ - 36
Bảo vệ người tiêu dùng - 37
Thị trường tiêu dùng - 32
Nông thôn mới - 38
Dân tộc thiểu số và miền núi - 39
Nội dung chuyên đề - 36


## Lưu dữ liệu

Nếu bạn chạy ở máy cá nhân thì không cần, nhưng nếu mà chạy ở Colab thì nên lưu dữ liệu vào trong Google Drive


In [ ]:
# For Google Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set to your folder
FOLDER_SAVED_GOOGLE_COLAB = "/content/drive/MyDrive/crawl-news/"

# Copy
!cp -r data $FOLDER_SAVED_GOOGLE_COLAB